In [1]:
%pip install azure-ai-ml
%pip install azure-identity
%pip install datasets
%pip install mlflow
%pip install azureml-mlflow
%pip install fsspec

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

  Using cached azure_storage_blob-12.24.0-py3-none-any.whl.metadata (26 kB)
Using cached azure_storage_blob-12.24.0-py3-none-any.whl (408 kB)
  Attempting uninstall: azure-storage-blob
    Found existing installation: azure-storage-blob 12.19.0
    Uninstalling azure-storage-blob-12.19.0:
      Successfully uninstalled azure-storage-blob-12.19.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
azureml-mlflow 1.59.0.post1 requires azure-storage-blob<=12.19.0,>=12.5.0, but you have azure-storage-blob 12.24.0 which is incompatible.


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
  Using cached azure_storage_blob-12.19.0-py3-none-any.whl.metadata (26 kB)
Using cached azure_storage_blob-12.19.0-py3-none-any.whl (394 kB)
  Attempting uninstall: azure-storage-blob
    Found existing installation: azure-storage-blob 12.24.0
    Uninstalling azure-storage-blob-12.24.0:
      Successfully uninstalled azure-storage-blob-12.24.0
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
azure-storage-file-datalake 12.18.0 requires azure-storage-blob>=12.24.0, but you have azure-storage-blob 12.19.0 which is incompatible.


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [12]:
from utils import NLIHuggingFaceInputDataset

# We can define train and test sample sizes here.
train_sample_size = 5
val_sample_size = 5
test_sample_size = 5

# Sample notebook using the dataset: https://huggingface.co/datasets/cestwc/conjnli
dataset_name = "cestwc/conjnli"
input_dataset = NLIHuggingFaceInputDataset()

# Note: train_split_name and test_split_name can vary by dataset. They are passed as arguments in load_hf_dataset.
# If val_split_name is None, the below function will split the train set to create the specified sized validation set.
train, val, test = input_dataset.load_hf_dataset(
    dataset_name=dataset_name,
    train_sample_size=train_sample_size,
    val_sample_size=val_sample_size,
    test_sample_size=test_sample_size,
    train_split_name="adversarial",
    val_split_name=None,
    test_split_name="dev",
)

print("Len of train data sample is " + str(len(train)))
print("Len of validation data sample is " + str(len(val)))
print("Len of test data sample is " + str(len(test)))


Len of train data sample is 5
Len of validation data sample is 5
Len of test data sample is 5


In [13]:
train[4]

{'premise': "A mother's boyfriend is a man whose girlfriend has a son or daughter from a previous marriage or relationship.",
 'hypothesis': "A mother's boyfriend is a man whose girlfriend has a girl or daughter from a previous marriage or relationship.",
 'label': 1}

In [14]:
# An example of how a final NLI prompt looks like
from utils import NLIPromptGenerator

# You can set the enable chain of thought flag to True to enable CoT prompting

nli_prompt_generator = NLIPromptGenerator(enable_chain_of_thought=True)
nli_prompt_generator.generate_prompt(train[0])

{'messages': [{'role': 'system',
   'content': "You are a helpful assistant. Write out in a step by step manner your reasoning about the answer using no more than 80 words. Based on the reasoning, produce the final answer. Your response should be in JSON format without using any backticks. The JSON is a dictionary whose keys are 'reason' and 'answer_choice'."},
  {'role': 'user',
   'content': "Given the following two texts, your task is to determine the logical relationship between them. The first text is the 'premise' and the second text is the 'hypothesis'. The relationship should be labeled as one of the following: 'entailment' if the premise entails the hypothesis, 'contradiction' if the premise contradicts the hypothesis, or 'neutral' if the premise neither entails nor contradicts the hypothesis.\n\nPremise: He was born in Illinois, but moved to Oklahoma in 1949.\nHypothesis: He was born in Illinois."}]}

In [15]:
from utils import AzureInference

url = "https://Meta-Llama-3-1-405B-Instruct-Ike.eastus2.models.ai.azure.com/chat/completions"
key = "mytx6i5Ci9rcf40wmR6820sJbGWefBtg"

az_llama_405b_model_inf = AzureInference(url=url, key=key)

In [16]:
from utils import NLISyntheticDatasetBuilder

nli_dataset_builder = NLISyntheticDatasetBuilder(
    nli_prompt_generator, inference_pointer=az_llama_405b_model_inf
)

# Write synthetic training and validation data to local directory.
nli_dataset_builder.build_dataset(train, file_name="train_nli")
nli_dataset_builder.build_dataset(val, file_name="valid_nli")

inferencing: https://Meta-Llama-3-1-405B-Instruct-Ike.eastus2.models.ai.azure.com/chat/completions
inferencing: https://Meta-Llama-3-1-405B-Instruct-Ike.eastus2.models.ai.azure.com/chat/completions
inferencing: https://Meta-Llama-3-1-405B-Instruct-Ike.eastus2.models.ai.azure.com/chat/completions
inferencing: https://Meta-Llama-3-1-405B-Instruct-Ike.eastus2.models.ai.azure.com/chat/completions
inferencing: https://Meta-Llama-3-1-405B-Instruct-Ike.eastus2.models.ai.azure.com/chat/completions
Write to file complete
inferencing: https://Meta-Llama-3-1-405B-Instruct-Ike.eastus2.models.ai.azure.com/chat/completions
inferencing: https://Meta-Llama-3-1-405B-Instruct-Ike.eastus2.models.ai.azure.com/chat/completions
inferencing: https://Meta-Llama-3-1-405B-Instruct-Ike.eastus2.models.ai.azure.com/chat/completions
inferencing: https://Meta-Llama-3-1-405B-Instruct-Ike.eastus2.models.ai.azure.com/chat/completions
inferencing: https://Meta-Llama-3-1-405B-Instruct-Ike.eastus2.models.ai.azure.com/chat